In [4]:
%load_ext autoreload
%autoreload 2

import requests
import json
import time
import re
from datetime import datetime
from unidecode import unidecode #pip install Unidecode
from nltk.tokenize import word_tokenize
from nltk.metrics import distance
from nltk.data import find

from actions.actions import id2names, normalize, get_closest_id, req_json, get_horarios, edit_distance

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
ori = get_closest_id('poli')
dst = get_closest_id('beni')
print(f'{id2names[ori][0]} -> {id2names[dst][0]}:\n')
print(get_horarios(ori, dst))

Universitat Politècnica -> Benimaclet:

Linea: 4     Hora: 04:47:00     Destino: Vicent Andrés Estellés 
Linea: 4     Hora: 05:10:00     Destino: Mas del Rosari 
Linea: 6     Hora: 05:22:00     Destino: Tossal del Rei 
Linea: 4     Hora: 05:27:00     Destino: Ll. Llarga - Terramelar 



In [20]:
print(edit_distance('absd','basa'))

0.5


In [33]:
import random

frases =  [
    "- Desde {origen} hasta {destino}",
    "- Desde {origen} a {destino}",
    "- De {origen} hasta {destino}",
    "- De {origen} a {destino}",
    "- Quiero ir de {origen} hasta {destino}",
    "- Quiero ir desde {origen} hasta {destino}",
    "- Quiero ir de {origen} a {destino}",
    "- Quiero ir desde {origen} a {destino}",
    "- Quiero ir de la estacion {origen} hasta {destino}",
    "- Quiero ir desde la estacion {origen} hasta {destino}",
    "- Quiero ir de la estacion {origen} a {destino}",
    "- Quiero ir desde la estacion {origen} a {destino}",
    "- {origen} a {destino}",
    "- Quiero ir desde la estacion {origen} a {destino}",
    "- Me gustaría ir desde {origen} hasta {destino}",
    "- Desde la estación {origen} hasta la estacion {destino}",
    "- Desde la estación {origen} a la estacion {destino}",
    "- De la estación {origen} hasta la estacion {destino}",
    "- De la estación {origen} a la estacion {destino}",
    "- Voy de {origen} hasta {destino}"
    "- Voy de {origen} a {destino}"
]

names = list(name2id.keys())

new_frases = []

for frase in frases:
    # pick 6 random stations
    random_stations = random.sample(names, 30)
    for i in range(0, len(random_stations), 2):
        estacion_origen = random_stations[i]
        estacion_destino = random_stations[i+1]
        origen = f'[{estacion_origen}]{{"entity": "station", "role": "departure"}}' 
        destino = f'[{estacion_destino}]{{"entity": "station", "role": "destination"}}' 
        new_frases.append(frase.format(origen=origen, destino=destino))

for frase in new_frases:
    print(frase)



- Desde [Tossal del Rei]{"entity": "station", "role": "departure"} hasta [Vicente Zaragozá]{"entity": "station", "role": "destination"}
- Desde [Jesús]{"entity": "station", "role": "departure"} hasta [Faitanar]{"entity": "station", "role": "destination"}
- Desde [Moncada - Alfara]{"entity": "station", "role": "departure"} hasta [Quart de Poblet]{"entity": "station", "role": "destination"}
- Desde [Francesc Cubells]{"entity": "station", "role": "departure"} hasta [L'Eliana]{"entity": "station", "role": "destination"}
- Desde [Russafa]{"entity": "station", "role": "departure"} hasta [Godella]{"entity": "station", "role": "destination"}
- Desde [Realón]{"entity": "station", "role": "departure"} hasta [La Cova]{"entity": "station", "role": "destination"}
- Desde [Aragó]{"entity": "station", "role": "departure"} hasta [Sant Ramon]{"entity": "station", "role": "destination"}
- Desde [Alberic]{"entity": "station", "role": "departure"} hasta [La Pobla de Farnals]{"entity": "station", "role": "